In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

load_dotenv("/Users/reejungkim/Documents/Git/working-in-progress/.env")

# 1. LLM 모델 로드 (Gemini Pro 사용)
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# 2. 임베딩 모델 로드
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# 3. 문서 로드 및 분할 (예시: PDF)
loader = PyPDFLoader("your_document.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# 4. 벡터 저장소 생성 (FAISS 사용)
vectorstore = FAISS.from_documents(texts, embeddings)

# 5. RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff" # "stuff", "map_reduce", "refine" 등 상황에 맞게 선택
)

# 6. 질문 실행
question = "문서의 핵심 내용은 무엇인가요?"
result = qa_chain.invoke({"query": question})
print(result['result'])